# Convolutional neural networks

In this unit we will learn about **Convolutional Neural Networks (CNNs)**, which are specifically designed for computer vision.

Computer vision is different from generic classification, because when we are trying to find a certain object in the picture, we are scanning the image looking for some specific **patterns** and their combinations. For example, when looking for a cat, we first may look for horizontal lines, which can form whiskers, and then certain combination of whiskers can tell us that it is actually a picture of a cat. Relative position and presence of certain patterns is important, and not their exact position on the image. 

To extract patterns, we will use the notion of **convolutional filters**. But first, let us load all dependencies and functions that we have defined in the previous units.

# 합성곱 신경망

이 단원에서는 컴퓨터 비전을 위해 특별히 설계된 **합성곱 신경망(CNN)**에 대해 배울 것입니다.

컴퓨터 비전은 일반적인 분류와 다릅니다. 왜냐하면 우리는 사진에서 특정한 물체를 찾으려고 할 때 특정한 **패턴**와 그 조합을 찾기 위해 이미지를 스캔하기 때문입니다. 예를 들어, 우리는 고양이를 찾을 때 수염을 형성할 수 있는 수평선을 먼저 찾을 수 있고, 그 다음 특정한 수염의 조합이 그것이 실제로 고양이의 사진임을 말해 줄 수 있습니다. 특정한 패턴의 상대적인 위치와 존재는 이미지에서 그들의 정확한 위치가 아니라 중요합니다.

패턴을 추출하기 위해 **convolutional filters**의 개념을 사용할 것입니다. 그러나 먼저 이전 단위에서 정의한 모든 종속성과 함수를 로드합니다.

In [ ]:
!wget https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/computer-vision-pytorch/pytorchcv.py
%pip install torchvision
%pip install torchinfo
%pip install pytorchcv

import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np
import pytorchcv

from pytorchcv import load_mnist, train, plot_results, plot_convolution, display_dataset
load_mnist(batch_size=128)

## Convolutional filters

Convolutional filters are small windows that run over each pixel of the image and compute weighted average of the neighboring pixels.



They are defined by matrices of weight coefficients. Let's see the examples of applying two different convolutional filters over our MNIST handwritten digits.

The vertical edge filter emphasizes changes in intensity that occur vertically across the image, making it useful for detecting vertical lines and edges.

## 컨벌루션 필터

컨볼루션 필터는 이미지의 각 픽셀을 실행하고 주변 픽셀의 가중 평균을 계산하는 작은 창입니다.



가중치 계수의 행렬로 정의됩니다. MNIST 손으로 쓴 숫자 위에 두 개의 다른 컨볼루션 필터를 적용하는 예를 살펴보겠습니다.

수직 에지 필터는 이미지 전반에 걸쳐 수직으로 발생하는 강도 변화를 강조하여 수직선 및 에지를 감지하는 데 유용합니다.

In [ ]:
plot_convolution(torch.tensor([[-1.,0.,1.],[-1.,0.,1.],[-1.,0.,1.]]),'Vertical edge filter')
plot_convolution(torch.tensor([[-1.,-1.,-1.],[0.,0.,0.],[1.,1.,1.]]),'Horizontal edge filter')


First filter is called a **vertical edge filter**, and it is defined by the following matrix:
$$
\left(
    \begin{matrix}
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
    \end{matrix}
\right)
$$
When this filter goes over relatively uniform pixel field, all values add up to 0. However, when it encounters a vertical edge in the image, high spike value is generated. That's why in the images above you can see vertical edges represented by high and low values, while horizontal edges are averaged out.

An opposite thing happens when we apply horizontal edge filter - horizontal lines are amplified, and vertical are averaged out.

In classical computer vision, multiple filters were applied to the image to generate features, which then were used by machine learning algorithm to build a classifier. However, in deep learning we construct networks that **learn** best convolutional filters to solve classification problem.

To do that, we introduce **convolutional layers**.

첫 번째 필터를 **vertical edge filter**라고 하며, 이 필터는 다음 행렬로 정의됩니다:
$$
\left(
    \begin{matrix}
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
    \end{matrix}
\right)
$$
이 필터가 비교적 균일한 픽셀 필드 위를 통과하면 모든 값이 0이 됩니다. 그러나 이미지에서 수직 에지를 만나면 높은 스파이크 값이 생성됩니다. 그렇기 때문에 위의 이미지에서 수평 에지가 평균화되는 동안 높은 값과 낮은 값으로 표시되는 수직 에지를 볼 수 있습니다.

수평 에지 필터를 적용하면 반대의 일이 발생합니다 - 수평 라인이 증폭되고 수직이 평균화됩니다.

고전적인 컴퓨터 비전에서는 여러 필터를 이미지에 적용하여 특징을 생성한 다음 기계 학습 알고리즘이 분류기를 구축하는 데 사용했습니다. 그러나 딥 러닝에서는 분류 문제를 해결하기 위해 **학습** 최고의 컨볼루션 필터를 사용하는 네트워크를 구성합니다.

이를 위해 **컨볼루션 레이어**를 도입합니다.

## Covolutional layers

Convolutional layers are defined using `nn.Conv2d` construction. We need to specify the following:
* `in_channels` - number of input channels. In our case we are dealing with a grayscale image, thus number of input channels is 1.
* `out_channels` - number of filters to use. We will use 9 different filters, which will give the network plenty of opportunities to explore which filters work best for our scenario.
* `kernel_size` is the size of the sliding window. Usually 3x3 or 5x5 filters are used.

Simplest CNN will contain one convolutional layer. Given the input size 28x28, after applying nine 5x5 filters we will end up with a tensor of 9x24x24 (the spatial size is smaller, because there are only 24 positions where a sliding interval of length 5 can fit into 28 pixels).

After convolution, we flatten 9x24x24 tensor into one vector of size 5184, and then add linear layer, to produce 10 classes. We also use `relu` activation function in between layers. 

The Rectified Linear Unit (ReLU) activation function is one of the most commonly used activation functions in neural networks, especially in deep learning models. The function is defined mathematically as:

ReLU(x)=max(0,x)

Here’s what this means:

If x is greater than 0, the function returns x.
If x is less than or equal to 0, the function returns 0.

Properties of ReLU
Non-linear: While it looks like a linear function, ReLU introduces a non-linearity (a simple threshold at 0), which allows models to learn more complex patterns.
Computationally Efficient: It is very efficient to compute as it only requires checking if the input is positive or not.
Sparse Activation: In practice, ReLU results in sparse activations; i.e., only a subset of neurons in a layer are active at a given time.

## 코볼루셔널 레이어

컨벌루션 레이어는 'n.Conv2d' 구조를 사용하여 정의합니다. 다음을 지정해야 합니다:
* in_channels - 입력 채널의 수. 우리의 경우 그레이스케일 이미지를 다루고 있으므로 입력 채널의 수는 1입니다.
* out_channels - 사용할 필터 수. 우리는 9개의 다른 필터를 사용할 것이고, 이것은 네트워크가 우리의 시나리오에 가장 적합한 필터를 탐색할 수 있는 충분한 기회를 줄 것입니다.
* kernel_size는 슬라이딩 윈도우의 크기입니다. 보통 3x3 또는 5x5 필터를 사용합니다.

가장 간단한 CNN은 하나의 컨볼루션 레이어를 포함할 것입니다. 입력 크기 28x28이 주어지면, 9개의 5x5 필터를 적용한 후, 우리는 결국 9x24x24의 텐서(길이 5의 슬라이딩 간격이 28개의 픽셀로 들어갈 수 있는 위치가 24개뿐이기 때문에 공간 크기가 더 작습니다)로 끝납니다.

컨볼루션 후에는 9x24x24 텐서를 5184 크기의 벡터 하나로 평평하게 만든 다음 선형 레이어를 추가하여 10개의 클래스를 생성합니다. 레이어 사이의 'relu' 활성화 함수도 사용합니다. 

ReLU(Relectricated Linear Unit) 활성화 함수는 신경망, 특히 딥러닝 모델에서 가장 일반적으로 사용되는 활성화 함수 중 하나입니다. 함수는 수학적으로 다음과 같이 정의됩니다:

ReLU(x)=max(0,x)

이것은 다음을 의미합니다:

x가 0보다 크면 함수는 x를 반환합니다.
x가 0보다 작거나 같으면 함수는 0을 반환합니다.

ReLU의 특성
비선형: 선형 함수처럼 보이지만 ReLU는 비선형성(0에서 간단한 임계값)을 도입하여 모델이 더 복잡한 패턴을 학습할 수 있습니다.
계산 효율성: 입력이 양인지 아닌지 확인하기만 하면 되므로 계산이 매우 효율적입니다.
희소 활성화: 실제로 ReLU는 희소 활성화를 초래합니다. 즉, 레이어에서 뉴런의 하위 집합만이 특정 시간에 활성화됩니다.

In [ ]:
import torch.nn as nn
%pip install torchsummary
from torchsummary import summary

class OneConv(nn.Module): # Defines a new class called OneConv that inherits from PyTorch's nn.Module. nn.Module is the base class for all neural network modules in PyTorch
    def __init__(self): # This method initializes the OneConv class. The __init__ method is a special method in Python classes. It gets called when an object of the class is created
        super(OneConv, self).__init__() # This line calls the constructor of the superclass (nn.Module). This is necessary to properly initialize the inherited class, setting up internal mechanisms that are crucial for the model's training and inference operations in PyTorch
        self.conv = nn.Conv2d(in_channels=1,out_channels=9,kernel_size=(5,5)) # a 2D convolutional layer is defined and assigned to self.conv
        self.flatten = nn.Flatten() # This creates an instance of the Flatten layer and assigns it to self.flatten. The Flatten layer converts a multi-dimensional input into a 1D array. This is typically used to transition from convolutional layers to fully connected layers
        self.fc = nn.Linear(5184,10) # This line defines a fully connected (or linear) layer that is assigned to self.fc. The layer transforms an input of 5184 features to 10 features.

    def forward(self, x): # The forward method defines the forward pass of the module. x is the input tensor that passes through the model
        if x.dim() == 5 and x.size(2) == 1:  # Check for unexpected extra dimension
            x = x.squeeze(2)
        x = nn.functional.relu(self.conv(x)) # This line applies the defined convolutional layer self.conv to the input x, then applies the ReLU activation function to the output of the convolution
        x = self.flatten(x) # This line applies the self.flatten layer to the output of the ReLU activation, converting all the feature maps into a single long vector, which can be fed into fully connected layers
        x = nn.functional.log_softmax(self.fc(x),dim=1) # Applies the fully connected layer self.fc to the flattened vector x. The result is then passed through a log softmax function. log_softmax is a logarithmic version of the softmax function, which is used to compute probabilities for multi-class classification problems. The dim=1 argument specifies that the softmax should be applied to the second dimension, which corresponds to the class probabilities for each input in the batch

        return x  # The final processed tensor x, which contains the log probabilities of the classes, is returned from the forward method. This output can be used by a loss function during training to compute the error and update the model weights

# Create an instance of the network
net = OneConv()

# Print the summary of the model
summary(net,input_size=(1,1,28,28))

You can see that this network contains around 50k trainable parameters, compared to around 80k in fully-connected multi-layered networks. This allows us to achieve good results even on smaller datasets, because convolutional networks generalize much better.

이 네트워크에는 완전히 연결된 다층 네트워크의 경우 약 80,000개에 비해 약 50,000개의 훈련 가능한 매개 변수가 포함되어 있음을 알 수 있습니다. 이를 통해 컨볼루션 네트워크가 훨씬 더 잘 일반화되기 때문에 더 작은 데이터 세트에서도 좋은 결과를 얻을 수 있습니다.

In [ ]:
import torch
from torch import optim
import torch.nn.functional as F

def train(model, train_loader, test_loader, epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': [], 'val_acc': [], 'val_loss': []}
    
    try:
        for epoch in range(epochs):
            model.train()
            train_loss, train_correct, train_total = 0, 0, 0
            for data, target in train_loader:
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(data)
                
                if output is None:
                    print("Warning: Model output is None.")
                    continue
                
                loss = criterion(output, target)
                if loss is None:
                    print("Warning: Loss computation returned None.")
                    continue
                
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                train_total += target.size(0)
                train_correct += (predicted == target).sum().item()
            
            train_loss /= len(train_loader.dataset)
            train_acc = 100. * train_correct / train_total
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            
            model.eval()
            test_loss, test_correct, test_total = 0, 0, 0
            with torch.no_grad():
                for data, target in test_loader:
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    if output is None:
                        print("Warning: Model output is None during evaluation.")
                        continue
                    
                    loss = criterion(output, target)
                    if loss is None:
                        print("Warning: Loss computation returned None during evaluation.")
                        continue
                    
                    test_loss += loss.item()
                    _, predicted = torch.max(output.data, 1)
                    test_total += target.size(0)
                    test_correct += (predicted == target).sum().item()
            
            test_loss /= len(test_loader.dataset)
            test_acc = 100. * test_correct / test_total
            history['test_loss'].append(test_loss)
            history['test_acc'].append(test_acc)
            
            print(f'Epoch {epoch+1}/{epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
    return history



In [ ]:
hist = train(net,train_loader,test_loader,epochs=5)
if hist is None:
    print("Training did not return any history.")
else:
    plot_results(hist)

We are able to achieve higher accuracy, and much faster, compared to the fully-connected networks.

We can also visualize the weights of our trained convolutional layers, to try and make some more sense of what is going on:

우리는 완전히 연결된 네트워크에 비해 더 높은 정확도와 훨씬 더 빠른 속도를 달성할 수 있습니다.

또한 훈련된 컨볼루션 레이어의 가중치를 시각화하여 무슨 일이 일어나고 있는지 좀 더 이해할 수 있습니다: